In [ ]:
#!/usr/bin/env python3
"""kobuki_ceiling_nav_single.py – ordered‑marker chase (no threads)
==================================================================
한 개의 ROS 노드 안에서
마커 **12 → 13 → 14 → 17 → 16 → 15** 를 차례대로 추종하여 /cmd_vel 을 내보낸다.

* **도달 판정** : 목표 마커가 카메라 중앙에 와서
  수평 거리 < 0.30 m & |yaw 오차| < 0.05 rad(≈ 3°)
* **속도 상한** : 선속 0.4 m/s, 각속 0.8 rad/s
"""
from __future__ import annotations

import math
import os
from typing import Dict, Tuple, List

import cv2
import cv2.aruco as aruco
import numpy as np
import rospy
from geometry_msgs.msg import Twist

# ------------------------------------------------------------------
# 파라미터 ----------------------------------------------------------
# ------------------------------------------------------------------

MARKER_LEN = 0.10
ARUCO_DICT = aruco.DICT_4X4_50

PATH_IDS: List[int] = [12, 13, 14, 17, 16, 15]  # 따라갈 순서

MARKER_WORLD: Dict[int, Tuple[float, float, float]] = {
    0:  (0.0, 0.0, 0.0),  1:  (1.5, 0.0, 0.0),
    2:  (3.0, 0.0, 0.0),  3:  (4.5, 0.0, 0.0),
    4:  (0.0, 1.5, 0.0),  5:  (1.5, 1.5, 0.0),
    6:  (3.0, 1.5, 0.0),  7:  (4.5, 1.5, 0.0),
    8:  (0.0, 3.0, 0.0),  9:  (1.5, 3.0, 0.0),
    10: (3.0, 3.0, 0.0), 11: (4.5, 3.0, 0.0),
    12: (0.7, 0.7, 0.0), 13: (2.3, 0.7, 0.0),
    14: (3.7, 0.7, 0.0), 15: (0.7, 2.3, 0.0),
    16: (2.3, 2.3, 0.0), 17: (3.7, 2.3, 0.0),
}

MARKER_YAW_DEG: Dict[int, float] = {
    0:   0,   1:   0,   2: -180, 3:   0,
    4: -180, 5: -180, 6:   0,  7: -180,
    8:   0,   9:   0,  10: -180, 11:   0,
    12: -180, 13:  -90, 14: -270, 15:   0,
    16:  -90, 17:  -90,
}

FIXED_Z = -1.9

LIN_KP = 0.8; LIN_MAX = 0.4
ANG_KP = 1.5; ANG_MAX = 0.8
GOAL_TOL = 0.30
YAW_TOL = 0.05

FRAME_W = 1280
FRAME_H = 720

# ------------------------------------------------------------------
# 보조 함수 ----------------------------------------------------------
# ------------------------------------------------------------------

def z_rot(deg: float) -> np.ndarray:
    rad = math.radians(deg)
    c, s = math.cos(rad), math.sin(rad)
    return np.array([[c, -s, 0], [s, c, 0], [0, 0, 1]])


def normalize(angle: float) -> float:
    while angle > math.pi:
        angle -= 2 * math.pi
    while angle < -math.pi:
        angle += 2 * math.pi
    return angle

# ------------------------------------------------------------------
# 메인 클래스 --------------------------------------------------------
# ------------------------------------------------------------------

class KobukiCeilingNav:
    def __init__(self):
        rospy.init_node('kobuki_ceiling_nav')
        self.cmd_pub = rospy.Publisher('/mobile_base/commands/velocity', Twist, queue_size=10)

        # 카메라 열기 --------------------------------------------------
        dev = rospy.get_param('~dev', 0)
        self.cap = cv2.VideoCapture(dev)
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_W)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_H)
        if not self.cap.isOpened():
            rospy.logfatal('camera open fail')
            raise RuntimeError('camera')

        # intrinsic ---------------------------------------------------
        base = os.path.dirname(os.path.abspath(__file__))
        K_path = os.path.join(base, 'calibration_matrix.npy')
        D_path = os.path.join(base, 'distortion_coefficients.npy')
        self.K = np.load(K_path)
        self.D = np.load(D_path)

        # ArUco -------------------------------------------------------
        self.aruco_dict = aruco.getPredefinedDictionary(ARUCO_DICT)
        self.aruco_param = aruco.DetectorParameters()

        # 경로 추종 ----------------------------------------------------
        self.idx = 0  # PATH_IDS 인덱스
        rospy.loginfo('path order: %s', PATH_IDS)

    # ----------------------------------------------------------------
    # 메인 루프 -------------------------------------------------------
    # ----------------------------------------------------------------
    def run(self):
        rate = rospy.Rate(20)
        while not rospy.is_shutdown() and self.idx < len(PATH_IDS):
            ret, frame = self.cap.read()
            if not ret:
                rate.sleep(); continue

            try:
                pose, ids = self.localize(frame)
            except RuntimeError:
                # 마커 하나도 못 봤으면 멈춤
                self.cmd_pub.publish(Twist())
                rate.sleep(); continue

            target_id = PATH_IDS[self.idx]
            if target_id not in ids:
                # 타깃 안 보이면 정지
                self.cmd_pub.publish(Twist())
                rate.sleep(); continue

            # 목표 점 -------------------------------------------------
            tx, ty, _ = MARKER_WORLD[target_id]
            dx = tx - pose[0]
            dy = ty - pose[1]
            dist = math.hypot(dx, dy)
            desired_yaw = math.atan2(dy, dx)
            yaw_err = normalize(desired_yaw - pose[2])

            # 도달 판정 ---------------------------------------------
            if dist < GOAL_TOL and abs(yaw_err) < YAW_TOL:
                rospy.loginfo('Reached marker %d', target_id)
                self.idx += 1
                self.cmd_pub.publish(Twist())
                rate.sleep(); continue

            # PID ----------------------------------------------------
            lin = max(-LIN_MAX, min(LIN_MAX, LIN_KP * dist))
            ang = max(-ANG_MAX, min(ANG_MAX, ANG_KP * yaw_err))

            cmd = Twist(); cmd.linear.x = lin; cmd.angular.z = ang
            self.cmd_pub.publish(cmd)
            rate.sleep()

        # 경로 완료 ----------------------------------------------------
        rospy.loginfo('All markers reached!')
        self.cmd_pub.publish(Twist())

    # ----------------------------------------------------------------
    # 위치 추정 -------------------------------------------------------
    # ----------------------------------------------------------------
    def localize(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = aruco.detectMarkers(gray, self.aruco_dict,
                                              parameters=self.aruco_param)
        if ids is None:
            raise RuntimeError('no markers')

        ids = ids.flatten()
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(
            corners, MARKER_LEN, self.K, self.D)

        pos_acc = np.zeros(2)
        yaw_acc = 0.0
        w_sum = 0.0
        visible = set()

        for rvec, tvec, mid in zip(rvecs, tvecs, ids):
            mid = int(mid)
            if mid not in MARKER_WORLD:
                continue
            visible.add(mid)
            # cam→marker
            R_cm, _ = cv2.Rodrigues(rvec)
            T_cm = np.eye(4)
            T_cm[:3, :3] = R_cm
            T_cm[:3, 3] = tvec.flatten()

            # world→marker
            wx, wy, wz = MARKER_WORLD[mid]
            T_wm = np.eye(4)
            T_wm[:3, :3] = z_rot(MARKER_YAW_DEG.get(mid, 0))
            T_wm[:3, 3] = [wx, wy, wz]

            T_wc = T_wm @ np.linalg.inv(T_cm)
            x, y = T_wc[0, 3], T_wc[1, 3]
            dist = math.hypot(x - wx, y - wy)
            w = 1.0 / (dist + 1e-6)
            pos_acc += w * np.array([x, y])
            yaw_acc += w * math.atan2(T_wc[1, 0], T_wc[0, 0])  # yaw
            w_sum += w

        if not visible:
            raise RuntimeError('no valid')

        x_avg, y_avg = pos_acc / w_sum
        yaw_avg = yaw_acc / w_sum
        return (x_avg, y_avg, yaw_avg), visible

# ------------------------------------------------------------------
if __name__ == '__main__':
    try:
        nav = KobukiCeilingNav()
        nav.run()
    except rospy.ROSInterruptException:
        pass
